In [ ]:
%pip install boto3
%pip install pillow
%pip install numpy
%pip install picamraw
%pip install tifffile

In [17]:
import os
import boto3
import botocore
from datetime import datetime
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import numpy as np
import picamraw
from tifffile import tifffile
from picamraw import PiRawBayer, PiCameraVersion


source_camera = "GONet058"

source_image_bucket = "gonet"
tiff_bucket = "gonet.split.tiff"
jpeg_bucket = "gonet.split.jpeg"
session = boto3.Session(profile_name='adler')
s3 = session.resource('s3')
s3_client = session.client("s3")
src_bucket = s3.Bucket(source_image_bucket)
key_list = []

for object_summary in src_bucket.objects.filter(Prefix=f"{source_camera}/"):
    key_list.append(object_summary.key)
    # print(object_summary.key) #* debug

print(f"Images in Bucket Prefix:: {len(key_list)}")

for image in key_list[40000:40020]:
    print(image)

    image_name = (f"{image.split('/')[1].split('.')[0]}")
    print(image_name)
    source_image_tmp = (f"/tmp/{image_name}.jpg")
    tiff_filename = (f"{image.split('/')[1].split('.')[0]}.tiff")
    jpeg_filename = (f"{image.split('/')[1].split('.')[0]}.jpeg")
    print(f"TIFF Filename:: {tiff_filename}")


# * Get the files
    s3_client.download_file(source_image_bucket, image, source_image_tmp)
    source_image = picamraw.PiRawBayer(
        filepath=source_image_tmp,  camera_version=picamraw.PiCameraVersion.V1, sensor_mode=0)
    c = source_image.to_rgb()
    array = (64*c.astype(np.uint16))
    tifffile.imwrite((f"/tmp/{image_name}.tiff"), array, photometric='rgb')

    s3_client.upload_file(
        (f"/tmp/{tiff_filename}"), tiff_bucket, (f"{source_camera}/{tiff_filename}"))
    # print('TIFF Uploaded')

    # Splitting off the JPEG, saving and applying EXIF from source to JPEG
    jpeg = Image.open(source_image_tmp).convert("RGB")
    exif = jpeg.info['exif']
    print('EXIF saved on JPEG')
    jpeg.save((f"/tmp/{jpeg_filename}"), 'JPEG', exif=exif)
    # print('JPEG Uploaded')
    s3_client.upload_file((f"/tmp/{jpeg_filename}"), jpeg_bucket,
                          (f"{source_camera}/{jpeg_filename}"))

    print("Cleaning up /tmp")
    os.remove(source_image_tmp)
    os.remove(f"/tmp/{tiff_filename}")
    os.remove(f"/tmp/{jpeg_filename}")


Images in Bucket Prefix:: 66843
GONet058/058_201013_063014_1602570678.jpg
058_201013_063014_1602570678
TIFF Filename:: 058_201013_063014_1602570678.tiff
EXIF saved on JPEG
Cleaning up /tmp
GONet058/058_201013_063014_1602570698.jpg
058_201013_063014_1602570698
TIFF Filename:: 058_201013_063014_1602570698.tiff
EXIF saved on JPEG
Cleaning up /tmp
GONet058/058_201013_063509_1602570910.jpg
058_201013_063509_1602570910
TIFF Filename:: 058_201013_063509_1602570910.tiff
EXIF saved on JPEG
Cleaning up /tmp
GONet058/058_201013_063509_1602570935.jpg
058_201013_063509_1602570935
TIFF Filename:: 058_201013_063509_1602570935.tiff
EXIF saved on JPEG
Cleaning up /tmp
GONet058/058_201013_063509_1602570954.jpg
058_201013_063509_1602570954
TIFF Filename:: 058_201013_063509_1602570954.tiff
EXIF saved on JPEG
Cleaning up /tmp
GONet058/058_201013_063509_1602570973.jpg
058_201013_063509_1602570973
TIFF Filename:: 058_201013_063509_1602570973.tiff
EXIF saved on JPEG
Cleaning up /tmp
GONet058/058_201013_063509